# Задание:

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

## Загрузка библиотек и модулей

In [1]:
#!g1.1
import tensorflow as tf
import numpy as np
import os
import time

## Загрузка и подготовка данных

In [2]:
#!g1.1
!ls *.txt

evgenyi_onegin.txt  рубаи.txt


In [3]:
#!g1.1
text = open('рубаи.txt', 'rb').read().decode(encoding='utf-8')
# Длина Евгения Онегина (кол-во символов)
print('Длина Рубаи: {} символов'.format(len(text)))

Длина Рубаи: 23660 символов


In [4]:
#!g1.1
print(text[:500])

Омар Хайям. Рубаи
Цель вечная движенья миров вселенной - мы.
В глазу рассудка ясном зрачок мгновенный - мы.
Похож на яркий перстень летящий круг миров.
На перстне этом быстром узор нетленный - мы.

Из всех ушедших в бесконечный путь
Сюда вернулся разве кто-нибудь?
Так в этом старом караван-сарае,
Смотри, чего-нибудь не позабудь.

Для достойного - нету достойных наград,
Я живот положить за достойного рад.
Хочешь знать, существуют ли адские муки?
Жить среди недостойных - вот истинный


In [5]:
#!g1.1
# Уникальные символы в тексте
vocab = sorted(set(text))
vocab_size = len(vocab)
print('{} уникальных символов в тексте'.format(vocab_size))

69 уникальных символов в тексте


In [6]:
#!g1.1
# Сопоставим уникальные символы текста и индексы
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
# Представим текст в виде последовательности чисел (индексов)
text_as_int = np.array([char2idx[c] for c in text])

In [7]:
#!g1.1
print(text_as_int[:200]), print(text[:200])

[25 49 37 53  2 31 37 46 68 49  7  2 27 56 38 37 45  1  0 32 42 48 65  2
 39 42 60 50 37 68  2 41 39 45 43 42 50 65 68  2 49 45 53 51 39  2 39 54
 42 48 42 50 50 51 46  2  6  2 49 64  7  1  0 14  2 40 48 37 44 56  2 53
 37 54 54 56 41 47 37  2 68 54 50 51 49  2 44 53 37 60 51 47  2 49 40 50
 51 39 42 50 50 64 46  2  6  2 49 64  7  1  0 26 51 58 51 43  2 50 37  2
 68 53 47 45 46  2 52 42 53 54 55 42 50 65  2 48 42 55 68 62 45 46  2 47
 53 56 40  2 49 45 53 51 39  7  1  0 24 37  2 52 42 53 54 55 50 42  2 66
 55 51 49  2 38 64 54 55 53 51 49  2 56 44 51 53  2 50 42 55 48 42 50 50
 64 46  2  6  2 49 64  7]
Омар Хайям. Рубаи
Цель вечная движенья миров вселенной - мы.
В глазу рассудка ясном зрачок мгновенный - мы.
Похож на яркий перстень летящий круг миров.
На перстне этом быстром узор нетленный - мы.


(None, None)

In [8]:
#!g1.1
len(text_as_int), len(text)

(23660, 23660)

### Подготовка

In [9]:
#!g1.1
# Максимальная длина предложения в символах для единичного ввода
seq_length = 100
# Кол-во примеров в эпоху
examples_per_epoch = len(text)//(seq_length+1)

# Размер батча
BATCH_SIZE = 64
# размер буфера для перемешивания данных
BUFFER_SIZE = 10000

# Размерность эмбеддинга
embedding_dim = 256

# Число ячеек
rnn_units = 1024

In [10]:
#!g1.1
# Создаем датасет
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
print(len(char_dataset))
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])
    print(i)

23660
О
tf.Tensor(25, shape=(), dtype=int64)
м
tf.Tensor(49, shape=(), dtype=int64)
а
tf.Tensor(37, shape=(), dtype=int64)
р
tf.Tensor(53, shape=(), dtype=int64)
 
tf.Tensor(2, shape=(), dtype=int64)


/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset
  warnings.warn(message)


In [11]:
#!g1.1
# Разобьём датасет на батчи длиной seq_length+1 символов, последний неполный батч отбрасываем
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Омар Хайям. Рубаи\r\nЦель вечная движенья миров вселенной - мы.\r\nВ глазу рассудка ясном зрачок мгновенн'
'ый - мы.\r\nПохож на яркий перстень летящий круг миров.\r\nНа перстне этом быстром узор нетленный - мы.\r\n'
'\r\nИз всех ушедших в бесконечный путь\r\nСюда вернулся разве кто-нибудь?\r\nТак в этом старом караван-сара'
'е,\r\nСмотри, чего-нибудь не позабудь.\r\n\r\nДля достойного - нету достойных наград,\r\nЯ живот положить за '
'достойного рад.\r\nХочешь знать, существуют ли адские муки?\r\nЖить среди недостойных - вот истинный ад!\r'


/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset, sequences
  warnings.warn(message)


In [12]:
#!g1.1
# Разобьём каждый батч на признак и целевую переменную (без первого символа)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# из всех батчей создаем датасет из признаков и целевых переменных
dataset = sequences.map(split_input_target)
print(f'Количество батчей в датасете: {len(dataset)}')

Количество батчей в датасете: 234


/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset, dataset, sequences
  warnings.warn(message)


Распечатаем первый пример входного текста и целевой переменной:

In [13]:
#!g1.1
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Омар Хайям. Рубаи\r\nЦель вечная движенья миров вселенной - мы.\r\nВ глазу рассудка ясном зрачок мгновен'
Target data: 'мар Хайям. Рубаи\r\nЦель вечная движенья миров вселенной - мы.\r\nВ глазу рассудка ясном зрачок мгновенн'


/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset, dataset, sequences
  warnings.warn(message)


In [14]:
#!g1.1
# Перемешаем разделенные данные
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset, dataset, sequences
  warnings.warn(message)


### Построение модели

In [15]:
#!g1.1
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
#!g1.1
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
#!g1.1
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           17664     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 69)            70725     
Total params: 30,513,477
Trainable params: 30,513,477
Non-trainable params: 0
____________________________________________

In [18]:
#!g1.1
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 69) # (batch_size, sequence_length, vocab_size)


/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset, dataset, sequences
  warnings.warn(message)


In [19]:
#!g1.1
example_batch_predictions[0]

<tf.Tensor: shape=(100, 69), dtype=float32, numpy=
array([[ 3.0124211e-06,  1.3225470e-05,  1.5613827e-05, ...,
        -1.1198234e-05, -2.4570701e-05,  8.0973095e-06],
       [ 1.8482799e-05,  5.5313812e-05,  3.2395070e-05, ...,
        -3.3656062e-05, -4.6851408e-05,  2.2662402e-05],
       [ 1.4125150e-05,  1.2846469e-04,  3.1863627e-05, ...,
        -6.0498904e-05, -4.5564473e-05,  3.4928624e-05],
       ...,
       [ 9.1013365e-04,  9.1484550e-04, -3.3225801e-03, ...,
         2.9508276e-03, -1.3989927e-03, -1.1057666e-03],
       [ 8.2392036e-04,  9.4673363e-04, -3.2138578e-03, ...,
         3.1240166e-03, -1.4995181e-03, -1.3588653e-03],
       [ 7.1854866e-04,  9.9469337e-04, -3.0948119e-03, ...,
         3.2871505e-03, -1.5599538e-03, -1.5439064e-03]], dtype=float32)>

In [20]:
#!g1.1
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
#!g1.1
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 'ед придут.\r\n\r\nДай вина! Здесь не место пустым словесам.\r\nПоцелуи любимой - мой хлеб и бальзам.\r\nГубы'

Next Char Predictions: 
 'ЛбщЛЧкаГ\rллнК"ЛСоКНВЗнаМОПжяшЯнжиптШс";ОЦ:хд?НЗГыыАшхепфБэч:Зр;бйЧ!Еаду!ВОгуЧЗфзХРПчРис?ШМЕбкъБАьлНБ'


### Обучение модели

In [22]:
#!g1.1
# т.к. задача многоклассовой классификации, то функция потерь - обычная кросс-энтропия
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [23]:
#!g1.1
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 69)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.234407


In [24]:
#!g1.1
# компиляция модели
model.compile(optimizer='adam', loss=loss)

#!g1.1
# Папка для сохранения чекпоинтов
checkpoint_dir = '/home/jupyter/work/resources/9_Языковое_моделирование/training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [25]:
#!g1.1
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta = 0.000001,
    patience = 10,
    verbose = 1,
    mode='auto',
    restore_best_weights=True)

In [26]:
#!g1.1
EPOCHS = 600

In [27]:
#!g1.1
# Обучение модели
history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stop])

Epoch 1/600
3/3 [==============================] - 5s 143ms/step - loss: 4.3264
Epoch 2/600
3/3 [==============================] - 0s 132ms/step - loss: 3.9630
Epoch 3/600
3/3 [==============================] - 0s 132ms/step - loss: 3.6634
Epoch 4/600
3/3 [==============================] - 0s 132ms/step - loss: 3.4485
Epoch 5/600
3/3 [==============================] - 0s 131ms/step - loss: 3.4045
Epoch 6/600
3/3 [==============================] - 0s 132ms/step - loss: 3.4031
Epoch 7/600
3/3 [==============================] - 0s 131ms/step - loss: 3.3903
Epoch 8/600
3/3 [==============================] - 0s 131ms/step - loss: 3.3808
Epoch 9/600
3/3 [==============================] - 0s 132ms/step - loss: 3.3857
Epoch 10/600
3/3 [==============================] - 0s 131ms/step - loss: 3.3801
Epoch 11/600
3/3 [==============================] - 0s 133ms/step - loss: 3.3783
Epoch 12/600
3/3 [==============================] - 0s 132ms/step - loss: 3.3752
Epoch 13/600
3/3 [===================

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: char_dataset, dataset, sequences
  warnings.warn(message)


### Генерация текста

In [31]:
#!g1.1
start_string=u"Цель вечная движенья миров вселенной - мы."

#!g1.1
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [34]:
#!g1.1
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           17664     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 69)            70725     
Total params: 30,513,477
Trainable params: 30,513,477
Non-trainable params: 0
____________________________________________

In [32]:
#!g1.1
# на вход процедуры поступает префикс start_string

def generate_text(model, start_string):
    num_generate = 500
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.5
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))

In [33]:
#!g1.1
text = generate_text(model, start_string=start_string)
print(text)

ValueError: Input 0 is incompatible with layer lstm: expected shape=(64, None, 256), found shape=(1, 42, 256)

In [ ]:
#!g1.1
len(text)